<a href="https://colab.research.google.com/github/jagjot29/Mini-Project/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok
!pip install joblib --quiet

In [ ]:
%%writefile knn_app.py
import streamlit as st
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import pickle
from PIL import Image
from sklearn.model_selection import GridSearchCV 
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

datadir = '/content/drive/MyDrive/Minor Project/Images'

categories = ['animals' , 'birds'] # Name the folder with these categories
target = []
flat_data = []
images = []

for i in categories:
  class_num = categories.index(i)
  path = os.path.join(datadir,i)

  for img in os.listdir(path):
    
    img_array = imread(os.path.join(path,img))

    img_resize = resize(img_array,(150,150,3)) # Value gets converted 0-1 from 0-255
    flat_data.append(img_resize.flatten())
    images.append(img_resize)
    target.append(class_num)
    

flat_data = np.array(flat_data)
target = np.array(target)
images = np.array(images)

x_train,x_test,y_train,y_test = train_test_split(flat_data,target,test_size=0.3,random_state=0)

param_grid = {
    'n_neighbors': [3,5,9,9],
    'weights': ['uniform', 'distance'],
    'metric':['identifier', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(),param_grid)
grid_results = grid.fit(x_train,y_train)


st.set_option('deprecation.showfileUploaderEncoding', False)
st.title('Image Classifier using KNN')
st.text('Upload the Image')


uploaded_file = st.file_uploader("Choose an image...",type="jpg")
if uploaded_file is not None :
  img = Image.open(uploaded_file)
  st.image(img,caption='Uploaded Image')

  if st.button('PREDICT') :
    st.write('Result...')
    CATEGORIES = ['animals' , 'birds']
    flat_data=[]
    img = np.array(img)
    img_resized = resize(img,(150,150,3))
    flat_data.append(img_resized.flatten())
    flat_data = np.array(flat_data)


    scaler = MinMaxScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(flat_data)


    
    model = KNeighborsClassifier()
    model.fit(x_train,y_train)
  
    y_pred = model.predict(x_test)



    y_predict = CATEGORIES[y_pred[0]]
    
    st.title(f' PREDICTED OUTPUT: {y_predict}')
    q = model.predict_proba(flat_data)

    for index, item in enumerate(CATEGORIES) :
      st.write(f'{item} : {q[0][index]*100}%')


In [ ]:
!nohup streamlit run knn_app.py &

url = ngrok.connect(port='8501')
url